In [139]:
import pandas as pd
from keras.layers import Input, Embedding, LSTM, Dense, Dot, Softmax, Concatenate, BatchNormalization, Attention
from keras.models import Model
import tensorflow as tf
import numpy as np

In [140]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
product_data = pd.read_pickle(basepath +'orders_and_products_by_customer.pkl')

In [141]:
product_data = product_data.sample(n=1000)
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
223249,8551990.0,[8551987],[[7593490]],[[Sneaker low Materialmix Leder grau]],[[1]],[[140.0]],[[2022-05-03 20:19:05]]
146632,6695858.0,[6695855],[[6113738]],[[Chelsea Boot Rauleder gelb]],[[1]],[[99.95]],[[2021-08-27 11:55:43]]
236694,8884452.0,[8884448],"[[7799189, 7824755, 7800504]]","[[Pantolette Glattleder schwarz, Pantolette Gl...","[[1, 1, 1]]","[[89.95, 89.95, 99.95]]","[[2022-06-13 06:58:14, 2022-06-13 06:58:14, 20..."
335206,10676713.0,[10676710],[[10230288]],[[Elegante Stiefelette Rauleder grün]],[[1]],[[120.0]],[[2023-02-02 17:13:40]]
84284,5213465.0,[5213463],[[4978647]],[[Sneaker low Rauleder blau]],[[1]],[[130.0]],[[2021-02-24 18:00:06]]
...,...,...,...,...,...,...,...
311371,10174420.0,[10174417],[[6596072]],[[Chelsea Boot Glattleder schwarz]],[[1]],[[89.0]],[[2022-11-25 04:59:02]]
181949,7523175.0,[7523172],"[[3843327, 6110647]]","[[Schnürstiefelette Rauleder blau, Schnürstief...","[[1, 1]]","[[69.0, 115.0]]","[[2021-11-23 22:35:38, 2021-11-23 22:35:38]]"
59353,4635721.0,[4635718],[[3503781]],[[Sneaker low Materialmix Leder braun]],[[1]],[[126.72]],[[2020-11-11 14:37:56]]
221614,8510699.0,[8510696],[[7595512]],[[Slipper Glattleder braun]],[[1]],[[140.0]],[[2022-04-29 12:31:49]]


In [142]:
product_data

,customer_id,order_ids,product_ids,product_names,amounts,total_prices,order_dates
223249,8551990.0,[8551987],[[7593490]],[[Sneaker low Materialmix Leder grau]],[[1]],[[140.0]],[[2022-05-03 20:19:05]]
146632,6695858.0,[6695855],[[6113738]],[[Chelsea Boot Rauleder gelb]],[[1]],[[99.95]],[[2021-08-27 11:55:43]]
236694,8884452.0,[8884448],"[[7799189, 7824755, 7800504]]","[[Pantolette Glattleder schwarz, Pantolette Gl...","[[1, 1, 1]]","[[89.95, 89.95, 99.95]]","[[2022-06-13 06:58:14, 2022-06-13 06:58:14, 20..."
335206,10676713.0,[10676710],[[10230288]],[[Elegante Stiefelette Rauleder grün]],[[1]],[[120.0]],[[2023-02-02 17:13:40]]
84284,5213465.0,[5213463],[[4978647]],[[Sneaker low Rauleder blau]],[[1]],[[130.0]],[[2021-02-24 18:00:06]]
...,...,...,...,...,...,...,...
311371,10174420.0,[10174417],[[6596072]],[[Chelsea Boot Glattleder schwarz]],[[1]],[[89.0]],[[2022-11-25 04:59:02]]
181949,7523175.0,[7523172],"[[3843327, 6110647]]","[[Schnürstiefelette Rauleder blau, Schnürstief...","[[1, 1]]","[[69.0, 115.0]]","[[2021-11-23 22:35:38, 2021-11-23 22:35:38]]"
59353,4635721.0,[4635718],[[3503781]],[[Sneaker low Materialmix Leder braun]],[[1]],[[126.72]],[[2020-11-11 14:37:56]]
221614,8510699.0,[8510696],[[7595512]],[[Slipper Glattleder braun]],[[1]],[[140.0]],[[2022-04-29 12:31:49]]


In [143]:
display(product_data[['customer_id', 'order_ids', 'product_ids', 'amounts', 'order_dates']])

,customer_id,order_ids,product_ids,amounts,order_dates
223249,8551990.0,[8551987],[[7593490]],[[1]],[[2022-05-03 20:19:05]]
146632,6695858.0,[6695855],[[6113738]],[[1]],[[2021-08-27 11:55:43]]
236694,8884452.0,[8884448],"[[7799189, 7824755, 7800504]]","[[1, 1, 1]]","[[2022-06-13 06:58:14, 2022-06-13 06:58:14, 20..."
335206,10676713.0,[10676710],[[10230288]],[[1]],[[2023-02-02 17:13:40]]
84284,5213465.0,[5213463],[[4978647]],[[1]],[[2021-02-24 18:00:06]]
...,...,...,...,...,...
311371,10174420.0,[10174417],[[6596072]],[[1]],[[2022-11-25 04:59:02]]
181949,7523175.0,[7523172],"[[3843327, 6110647]]","[[1, 1]]","[[2021-11-23 22:35:38, 2021-11-23 22:35:38]]"
59353,4635721.0,[4635718],[[3503781]],[[1]],[[2020-11-11 14:37:56]]
221614,8510699.0,[8510696],[[7595512]],[[1]],[[2022-04-29 12:31:49]]


In [144]:
# Define the model architecture
num_orders = sum(len(order) for order in product_data.order_ids)
num_products =  sum(len(order) for order in product_data.product_ids)
num_units = len([amount for order_amounts in product_data.amounts for amount in order_amounts])
num_prices = len([amount for order_amounts in product_data.total_prices for amount in order_amounts])
num_dates = len([amount for order_amounts in product_data.order_dates for amount in order_amounts])

print(num_orders, num_products, num_units, num_prices, num_dates)

1208 1208 1208 1208 1208


In [145]:
# Input layers
date_inputs = Input(shape=(num_dates))
# product_inputs = Input(shape=(num_orders, num_units))
product_inputs = Input(shape=(num_products))
# price_inputs = Input(shape=(num_orders, num_prices))

encoding_padding_mask = tf.math.logical_not(tf.math.equal(product_inputs, 0))

# Embedding layers
date_emb = Embedding(num_dates, 16, input_length=num_dates)(date_inputs)
product_emb = Embedding(num_products, 16, input_length=num_units)(product_inputs)

concat_embedding_input = Concatenate(
    name='concat_embedding_input')([date_emb, product_emb])

concat_embedding_input = BatchNormalization(
    name='batchnorm_inputs')(concat_embedding_input)

# LSTM layer
lstm = LSTM(64, return_sequences=True)(concat_embedding_input)

lstm = BatchNormalization(name='batchnorm_lstm')(lstm)

att = Attention(use_scale=False,
                                name='attention')(inputs=[lstm, lstm],
                                                  mask=[encoding_padding_mask,
                                                        encoding_padding_mask])


# Concatenate layer
#concat = Concatenate(axis=2)([context, price_inputs])

# Dense layer
output = Dense(1)(att)
# Define the model
model = Model(inputs=[date_inputs, product_inputs], outputs=output)
# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])

In [146]:
#utils.plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [147]:
# # Train the model
# X = np.asarray(product_data[['order_ids', 'product_ids']])
# y = np.asarray(product_data['amounts'])
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#
# X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
# y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)




In [149]:
arr = np.array([np.array(x).flatten() for x in product_data.product_ids])


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
#display(X_train)
#output

product_ids_flat = product_data.product_ids.to_numpy().reshape(-1)
order_dates_flat = product_data.order_dates.to_numpy().reshape(-1)

# Create the output variable
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)

# Create the dataset
dataset = tf.data.Dataset.from_tensor_slices((order_dates_flat, product_ids_flat, output_variable))

# Fit the model
history = model.fit(dataset, epochs=50, batch_size=32)


In [ ]:
display(order_dates_flat)

In [ ]:
y = []
output_variable = product_data.amounts.to_numpy().reshape(-1, 1)
dataset = tf.data.Dataset.from_tensor_slices((np.array(product_data.order_dates), np.array(product_data.product_ids), output_variable))
history = model.fit(dataset, epochs=50, batch_size=32)

In [ ]:
# Evaluate the model
loss, mse = model.evaluate([product_data.order_dates_test, product_data.product_ids_test, product_data.total_prices_test], product_data.total_prices_test)
print("Test loss: {:.4f}, Test MSE: {:.4f}".format(loss, mse))

In [ ]:
# split the data into training and testing sets
#X = pd.get_dummies(product_data['product__id'], prefix='product')
#y = product_data['amount']
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)